## Acquire process:

In [47]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt


# acquire
from env import host, user, password
from pydataset import data

In [3]:
# Create helper function to get the necessary connection url.

def get_connection(db_name):
    '''
    This function uses my info from my env file to
    create a connection url to access the Codeup db.
    '''
    from env import host, user, password
    return f'mysql+pymysql://{user}:{password}@{host}/{db_name}'


In [4]:
def get_telco_churn_data():
    '''
    This function reads in the iris data from the Codeup db
    and returns a pandas DataFrame with all columns.
    '''
    
    sql_query = '''
    SELECT *
    FROM customers
    JOIN contract_types ON customers.contract_type_id = contract_types.contract_type_id
    JOIN payment_types ON customers.payment_type_id = payment_types.payment_type_id
    JOIN internet_service_types ON customers.internet_service_type_id = internet_service_types.internet_service_type_id
    '''
    return pd.read_sql(sql_query, get_connection('telco_churn'))

_________

## Explore the data

In [6]:
#rename data to 'telco'
telco = get_telco_churn_data()

In [41]:
#look at data
telco.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id,contract_type,payment_type_id,payment_type,internet_service_type_id,internet_service_type
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,2,90.45,5957.9,No,3,Two year,2,Mailed check,1,DSL
1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,Yes,...,4,45.20,2460.55,No,3,Two year,4,Credit card (automatic),1,DSL
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,Yes,...,3,45.05,2560.1,No,3,Two year,3,Bank transfer (automatic),1,DSL
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,Yes,...,4,39.40,825.4,No,3,Two year,4,Credit card (automatic),1,DSL
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,No,...,3,85.15,6316.2,No,3,Two year,3,Bank transfer (automatic),1,DSL


In [9]:
#this data appears to have no nulls
#data has int and object data types
telco.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 27 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   customer_id               7043 non-null   object 
 1   gender                    7043 non-null   object 
 2   senior_citizen            7043 non-null   int64  
 3   partner                   7043 non-null   object 
 4   dependents                7043 non-null   object 
 5   tenure                    7043 non-null   int64  
 6   phone_service             7043 non-null   object 
 7   multiple_lines            7043 non-null   object 
 8   internet_service_type_id  7043 non-null   int64  
 9   online_security           7043 non-null   object 
 10  online_backup             7043 non-null   object 
 11  device_protection         7043 non-null   object 
 12  tech_support              7043 non-null   object 
 13  streaming_tv              7043 non-null   object 
 14  streamin

### data summary:
- 18 object data types
- 8 integer data types
- 1 float data type
- 0 null values


In [66]:
#change total charges from an object to a float- put that in PREPARE
telco.total_charges = telco.total_charges.str.replace(' ', '0').astype(float)

In [65]:
#descriptive statistics
telco.describe()

,senior_citizen,tenure,internet_service_type_id,contract_type_id,payment_type_id,monthly_charges,contract_type_id,payment_type_id,internet_service_type_id
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,1.872923,1.690473,2.315633,64.761692,1.690473,2.315633,1.872923
std,0.368612,24.559481,0.737796,0.833755,1.148907,30.090047,0.833755,1.148907,0.737796
min,0.000000,0.000000,1.000000,1.000000,1.000000,18.250000,1.000000,1.000000,1.000000
25%,0.000000,9.000000,1.000000,1.000000,1.000000,35.500000,1.000000,1.000000,1.000000
50%,0.000000,29.000000,2.000000,1.000000,2.000000,70.350000,1.000000,2.000000,2.000000
75%,0.000000,55.000000,2.000000,2.000000,3.000000,89.850000,2.000000,3.000000,2.000000
max,1.000000,72.000000,3.000000,3.000000,4.000000,118.750000,3.000000,4.000000,3.000000


_______

### get curious about the data

In [16]:
#we are trying to determine churn... look into that
telco.churn.value_counts()

#this shows 26.54% of customers churn (1869 out of 7043)

No     5174
Yes    1869
Name: churn, dtype: int64

In [15]:
#find ou how many are senior citizens and how many are not
telco.senior_citizen.value_counts()

#this shows that 16.21% of all customers churn (1142 out of 7043)

0    5901
1    1142
Name: senior_citizen, dtype: int64

In [12]:
#average tenure, max tenure, min tenure??
telco.tenure.mean(), telco.tenure.max(), telco.tenure.min()

(32.37114865824223, 72, 0)

In [13]:
#average monthly charges, max monthly charges, min monthly charges
telco.monthly_charges.mean(), telco.monthly_charges.max(), telco.monthly_charges.min()

(64.76169246059882, 118.75, 18.25)

In [64]:
#telco["churn"] = telco["churn"].astype(int)

In [60]:
#group by churn and senior_citizen 
senior_churn= telco.groupby(['churn','senior_citizen'])

In [61]:
#senior_churn = senior_churn.drop(['customer_id'], axis=1)
senior_churn.head()

,customer_id,gender,senior_citizen,partner,dependents,tenure,phone_service,multiple_lines,internet_service_type_id,online_security,...,payment_type_id,monthly_charges,total_charges,churn,contract_type_id,contract_type,payment_type_id,payment_type,internet_service_type_id,internet_service_type
0,0016-QLJIS,Female,0,Yes,Yes,65,Yes,Yes,1,Yes,...,2,90.45,5957.9,No,3,Two year,2,Mailed check,1,DSL
1,0017-DINOC,Male,0,No,No,54,No,No phone service,1,Yes,...,4,45.20,2460.55,No,3,Two year,4,Credit card (automatic),1,DSL
2,0019-GFNTW,Female,0,No,No,56,No,No phone service,1,Yes,...,3,45.05,2560.1,No,3,Two year,3,Bank transfer (automatic),1,DSL
3,0056-EPFBG,Male,0,Yes,Yes,20,No,No phone service,1,Yes,...,4,39.40,825.4,No,3,Two year,4,Credit card (automatic),1,DSL
4,0078-XZMHT,Male,0,Yes,No,72,Yes,Yes,1,No,...,3,85.15,6316.2,No,3,Two year,3,Bank transfer (automatic),1,DSL
20,0336-PIKEI,Male,1,Yes,No,72,Yes,No,1,Yes,...,3,74.40,5360.75,No,3,Two year,3,Bank transfer (automatic),1,DSL
44,0617-AQNWT,Female,0,Yes,No,64,No,No phone service,1,Yes,...,1,47.85,3147.5,Yes,3,Two year,1,Electronic check,1,DSL
47,0707-HOVVN,Female,1,No,No,70,Yes,Yes,1,No,...,3,75.50,5212.65,No,3,Two year,3,Bank transfer (automatic),1,DSL
91,1568-BEKZM,Male,1,Yes,No,70,Yes,Yes,1,Yes,...,4,90.05,6333.4,No,3,Two year,4,Credit card (automatic),1,DSL
108,2000-DHJUY,Female,1,Yes,No,70,Yes,Yes,1,Yes,...,3,92.20,6474.45,No,3,Two year,3,Bank transfer (automatic),1,DSL


_________